In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPooling2D, Input, BatchNormalization, DepthwiseConv2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from PIL import Image

# ===============================
# Load Dataset
# ===============================
BASE_DIR = r"C:\Users\Dishang\OneDrive\Desktop\UTKFace"

image_paths = []
age_labels = []
gender_labels = []

for filename in tqdm(os.listdir(BASE_DIR)):
    try:
        image_path = os.path.join(BASE_DIR, filename)
        temp = filename.split('_')
        age = int(temp[0])
        gender = int(temp[1])
        image_paths.append(image_path)
        age_labels.append(age)
        gender_labels.append(gender)
    except:
        continue

df = pd.DataFrame()
df['image'], df['age'], df['gender'] = image_paths, age_labels, gender_labels

gender_dict = {0: 'Male', 1: 'Female'}

# ===============================
# Feature Extraction
# ===============================
def extract_features_optimized(images, target_size=(48, 48)):
    features = []
    for image in tqdm(images):
        try:
            img = load_img(image, color_mode='rgb')
            img = img.resize(target_size, Image.Resampling.LANCZOS)
            img = np.array(img)
            features.append(img)
        except:
            continue
    features = np.array(features)
    return features

X = extract_features_optimized(df['image'], target_size=(48, 48))
print(f"Feature shape: {X.shape}")

# Normalize(#17_10)
X = X.astype(np.float32) / 255.0
X = X * 2.0 - 1.0   # scale to [-1, 1] for INT8 quantization


# Labels
y_gender = np.array(df['gender'][:len(X)])

# Split
X_train, X_test, y_gender_train, y_gender_test = train_test_split(
    X, y_gender, test_size=0.2, random_state=42, stratify=y_gender
)

print(f"Training set: {X_train.shape}, Test set: {X_test.shape}")

# ===============================
# Model Architecture
# ===============================
def create_high_accuracy_model():
    inputs = Input((48,48,3))

    # Conv Blocks
    x = DepthwiseConv2D((3,3), padding='same', activation='relu')(inputs)
    x = Conv2D(64, (1,1), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D()(x)

    x = DepthwiseConv2D((3,3), padding='same', activation='relu')(x)
    x = Conv2D(128, (1,1), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D()(x)

    x = DepthwiseConv2D((3,3), padding='same', activation='relu')(x)
    x = Conv2D(256, (1,1), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D()(x)

    x = GlobalAveragePooling2D()(x)

    # Dense Layers
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)

    outputs = Dense(1, activation='sigmoid', name='gender_out')(x)
    return Model(inputs, outputs)


model = create_high_accuracy_model()


# Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()
total_params = model.count_params()
print(f"\nTotal Parameters: {total_params:,} ({total_params/100000:.1f} lakh)")

# ===============================
# Callbacks
# ===============================
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        mode='min'
    ),
    ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        mode='max'
    )
]

# ===============================
# Training
# ===============================
history = model.fit(
    X_train, y_gender_train,
    batch_size=64,
    epochs=50,
    validation_data=(X_test, y_gender_test),
    callbacks=callbacks,
    verbose=1
)

print(f"Training completed! Total epochs run: {len(history.history['loss'])}")

# ===============================
# Evaluation
# ===============================
loss, acc = model.evaluate(X_test, y_gender_test, verbose=0)
print(f"\nTest Accuracy: {acc:.4f} ({acc*100:.2f}%)")

# ===============================
# Plot History
# ===============================
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title("Training & Validation Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# ===============================
# Sample Predictions
# ===============================
def test_predictions(model, X_test, y_gender_test, num_samples=5):
    indices = np.random.choice(len(X_test), num_samples, replace=False)
    plt.figure(figsize=(15, 3*num_samples))

    for i, idx in enumerate(indices):
        pred = model.predict(X_test[idx:idx+1], verbose=0)
        pred_gender = round(pred[0][0])
        conf = pred[0][0]

        plt.subplot(num_samples, 1, i+1)
        plt.imshow(X_test[idx])
        plt.title(f"True: {gender_dict[y_gender_test[idx]]} | Pred: {gender_dict[pred_gender]} | Conf: {conf:.3f}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

print("\nSample Predictions:")
test_predictions(model, X_test, y_gender_test)

# ===============================
# Final Summary
# ===============================
final_train_acc = max(history.history['accuracy'])
final_val_acc = max(history.history['val_accuracy'])

print(f"\n{'='*50}")
print(f"MODEL PERFORMANCE SUMMARY")
print(f"{'='*50}")
print(f"Total Parameters: {total_params:,} ({total_params/100000:.1f} lakh)")
print(f"Best Training Accuracy: {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
print(f"Best Validation Accuracy: {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
print(f"Test Accuracy: {acc:.4f} ({acc*100:.2f}%)")
print(f"{'='*50}")


  0%|          | 0/23710 [00:00<?, ?it/s]

  0%|          | 0/23709 [00:00<?, ?it/s]

Feature shape: (23709, 48, 48, 3)
Training set: (18967, 48, 48, 3), Test set: (4742, 48, 48, 3)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 48, 48, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ depthwise_conv2d (DepthwiseConv2D)   │ (None, 48, 48, 3)           │              30 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 48, 48, 64)          │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 48, 48, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 24, 24, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ depthwise_conv2d_1 (DepthwiseConv2D) │ (None, 24, 24, 64)          │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 24, 24, 128)         │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 24, 24, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ depthwise_conv2d_2 (DepthwiseConv2D) │ (None, 12, 12, 128)         │           1,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 256)         │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 12, 12, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 256)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gender_out (Dense)                   │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 144,159 (563.12 KB)

 Trainable params: 143,263 (559.62 KB)

 Non-trainable params: 896 (3.50 KB)


Total Parameters: 144,159 (1.4 lakh)
Epoch 1/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 46s 144ms/step - accuracy: 0.7103 - loss: 0.5588 - val_accuracy: 0.5228 - val_loss: 0.6907 - learning_rate: 0.0010
Epoch 2/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 42s 142ms/step - accuracy: 0.8118 - loss: 0.4171 - val_accuracy: 0.8248 - val_loss: 0.4464 - learning_rate: 0.0010
Epoch 3/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 524s 2s/step - accuracy: 0.8417 - loss: 0.3585 - val_accuracy: 0.7676 - val_loss: 0.4624 - learning_rate: 0.0010
Epoch 4/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 46s 156ms/step - accuracy: 0.8560 - loss: 0.3359 - val_accuracy: 0.7950 - val_loss: 0.4795 - learning_rate: 0.0010
Epoch 5/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 47s 158ms/step - accuracy: 0.8675 - loss: 0.3099 - val_accuracy: 0.8484 - val_loss: 0.3432 - learning_rate: 0.0010
Epoch 6/50
297/297 ━━━━━━━━━━━━━━━━━━━━ 42s 142ms/step - accuracy: 0.8727 - loss: 0.2937 - val_accuracy: 0.8568 - val_loss: 0.3238 - learning_rate: 0.0010
Epoch 7/50
297/297 ━━━━━━━━━━━━━━━

In [18]:
# Convert to fully quantized TFLite (INT8)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Representative dataset for calibration (take few training samples)
def representative_dataset():
    for i in range(100):
        img = X_train[i].astype(np.float32)
        img = np.expand_dims(img, axis=0)
        yield [img]

converter.representative_dataset = representative_dataset

# Force full int8 quantization
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert
tflite_quant_model = converter.convert()

# Save the smaller model
with open("gender_model_int8.tflite", "wb") as f:
    f.write(tflite_quant_model)

print("✅ INT8 quantized model saved successfully!")

# Check size
size = os.path.getsize("gender_model_int8.tflite") / 1024
print(f"Model size: {size:.2f} KB")


INFO:tensorflow:Assets written to: C:\Users\Dishang\AppData\Local\Temp\tmp3s43aqg9\assets


INFO:tensorflow:Assets written to: C:\Users\Dishang\AppData\Local\Temp\tmp3s43aqg9\assets


Saved artifact at 'C:\Users\Dishang\AppData\Local\Temp\tmp3s43aqg9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 48, 48, 3), dtype=tf.float32, name='keras_tensor_19')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2155835715856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2155835715280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2155835714128: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2155835716816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2155835717968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2155835714704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2155835715664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2155835714896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2155835715472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2155835716048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  21

In [19]:
import cv2
import numpy as np
import tensorflow as tf

# ===============================
# Load the INT8 TFLite model
# ===============================
interpreter = tf.lite.Interpreter(model_path="gender_model_int8.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Input shape (e.g., [1, 48, 48, 3])
input_shape = input_details[0]['shape']
img_height, img_width = input_shape[1], input_shape[2]

# ===============================
# Helper: Predict gender
# ===============================
def predict_gender_tflite(face_img):
    # Resize and normalize
    face_img = cv2.resize(face_img, (img_width, img_height))
    face_img = face_img.astype(np.float32) / 255.0

    # Quantize input to int8 if model expects int8
    if input_details[0]['dtype'] == np.int8:
        input_scale, input_zero_point = input_details[0]['quantization']
        face_img = face_img / input_scale + input_zero_point
        face_img = face_img.astype(np.int8)

    face_img = np.expand_dims(face_img, axis=0)

    interpreter.set_tensor(input_details[0]['index'], face_img)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])[0]

    # Dequantize output if int8
    if output_details[0]['dtype'] == np.int8:
        output_scale, output_zero_point = output_details[0]['quantization']
        output_data = (output_data.astype(np.float32) - output_zero_point) * output_scale

    pred = output_data[0]
    gender_label = "Female" if pred >= 0.5 else "Male"
    confidence = pred if pred >= 0.5 else 1 - pred
    return gender_label, float(confidence)

# ===============================
# Load Haar Cascade for Face Detection
# ===============================
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# ===============================
# Real-Time Webcam Feed
# ===============================
cap = cv2.VideoCapture(0)  # Change index if using external camera

print("Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        gender, conf = predict_gender_tflite(face)

        label = f"{gender} ({conf*100:.1f}%)"
        color = (0, 255, 0) if gender == "Male" else (255, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, cv2.LINE_AA)

    cv2.imshow("Gender Classification (INT8 Model)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Press 'q' to quit.


In [16]:
#17_10
import os

# Path to your quantized TFLite model
tflite_model_path = "gender_model_int8.tflite"

# Read the TFLite model as bytes
with open(tflite_model_path, "rb") as f:
    tflite_model = f.read()

# Output header file name
header_file = "gender_model_int8.h"

# Variable name for model data in C
var_name = os.path.splitext(os.path.basename(header_file))[0]

# Create the header file
with open(header_file, "w") as f:
    f.write("// =====================================================\n")
    f.write("//  TensorFlow Lite INT8 Model for Arduino Nano 33 BLE\n")
    f.write("//  Automatically generated header file\n")
    f.write("// =====================================================\n\n")

    f.write(f"#ifndef {var_name.upper()}_H_\n")
    f.write(f"#define {var_name.upper()}_H_\n\n")

    # Model length
    f.write(f"unsigned int {var_name}_len = {len(tflite_model)};\n")
    f.write(f"const unsigned char {var_name}[] = {{\n")

    # Write model bytes in hex format
    for i, byte in enumerate(tflite_model):
        if i % 12 == 0:
            f.write("\n ")
        f.write(f"0x{byte:02x}, ")

    f.write("\n};\n\n")
    f.write(f"#endif // {var_name.upper()}_H_\n")

print(f"✅ Header file '{header_file}' created successfully!")
print(f"📦 Model size: {len(tflite_model)/1024:.2f} KB")


✅ Header file 'gender_model_int8.h' created successfully!
📦 Model size: 175.77 KB


In [2]:
import cv2

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Resize function to match model input
def preprocess_face(face_img, target_size=(48,48)):
    face_img = cv2.resize(face_img, target_size, interpolation=cv2.INTER_LANCZOS4)
    face_img = face_img / 255.0
    face_img = np.expand_dims(face_img, axis=0)
    return face_img

# Function to detect faces and predict gender
def detect_and_predict_gender(image, model):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30,30))

    for (x,y,w,h) in faces:
        face_img = image[y:y+h, x:x+w]
        face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)  # convert to RGB
        face_input = preprocess_face(face_rgb, target_size=(48,48))
        
        pred = model.predict(face_input, verbose=0)[0][0]
        gender_label = "Female" if pred >= 0.5 else "Male"
        confidence = pred if pred >= 0.5 else 1 - pred

        # Draw rectangle and label
        cv2.rectangle(image, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(image, f"{gender_label} ({confidence*100:.1f}%)", 
                    (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
    
    return image

# --- Example: Real-time Webcam Gender Detection ---
cap = cv2.VideoCapture(0)  # 0 for default webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    output_frame = detect_and_predict_gender(frame, model)
    cv2.imshow("Gender Detection", output_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

cap.release()
cv2.destroyAllWindows()


In [3]:
# Save the trained model in HDF5 format
model.save("optimized_17_10_gender_model.h5")


In [4]:
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Enable optimization for size and speed
tflite_model = converter.convert()
# Save the TFLite model
with open("gender_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved successfully!")


INFO:tensorflow:Assets written to: C:\Users\Dishang\AppData\Local\Temp\tmpf7trcng6\assets


INFO:tensorflow:Assets written to: C:\Users\Dishang\AppData\Local\Temp\tmpf7trcng6\assets


Saved artifact at 'C:\Users\Dishang\AppData\Local\Temp\tmpf7trcng6'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 48, 48, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2152287177040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2152287177616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2152287177808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2152287177424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2152287179344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2152287176656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2152287178384: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2152287178960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2152287179920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2152287179728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  21522

In [5]:
def hex_to_c_array(hex_data, var_name):
    c_str=''


    c_str += '#ifndef' + var_name.upper() + '_H\n'
    c_str += '#define' + var_name.upper() + '_H\n\n'

    c_str+='\nunsigned int' +var_name + '_len =' +str(len(hex_data))+ ';\n'

    c_str+='unsigned char'+ var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data):

        hex_str = format(val, '#04x')

        if (i+1)<len(hex_data):
            hex_str+=','
        if (i+1)%12 ==0:
            hex_str+='\n'
        hex_array.append(hex_str)

        c_str+= '\n'+format (''.join(hex_array))+ '\n};\n\n'

        c_str+= '#endif //' + var_name.upper()+'_H'

        return c
        

In [6]:
# Load TFLite model
interpreter = tf.lite.Interpreter(model_path="gender_model.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to predict gender using TFLite
def predict_gender_tflite(face_img):
    # face_img should be RGB, normalized, shape = (48,48,3) or (64,64,3) depending on your model
    input_shape = input_details[0]['shape']
    face_img = cv2.resize(face_img, (input_shape[1], input_shape[2]))
    face_img = face_img / 255.0
    face_img = np.expand_dims(face_img, axis=0).astype(np.float32)

    interpreter.set_tensor(input_details[0]['index'], face_img)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_details[0]['index'])[0][0]

    gender_label = "Female" if pred >= 0.5 else "Male"
    confidence = pred if pred >= 0.5 else 1 - pred
    return gender_label, confidence


In [7]:
import os

tflite_model_path = "gender_model.tflite"

# Get size in MB
model_size = os.path.getsize(tflite_model_path) / (1024 * 1024)
print(f"TFLite model size: {model_size:.2f} MB")


TFLite model size: 0.16 MB


In [8]:
tflite_model_path = "gender_model.tflite"


In [9]:
import os

folder_path = r"C:\Users\Dishang\Documents\Arduino\tiny_ml"
os.makedirs(folder_path, exist_ok=True)
print("Folder exists or created:", folder_path)


Folder exists or created: C:\Users\Dishang\Documents\Arduino\tiny_ml


In [10]:
import os

# Use the exact file name here
tflite_model_path = r"C:\Users\Dishang\Documents\Arduino\tiny_ml\gender_model.tflite"

if not os.path.exists(tflite_model_path):
    raise FileNotFoundError(f"File not found: {tflite_model_path}")


FileNotFoundError: File not found: C:\Users\Dishang\Documents\Arduino\tiny_ml\gender_model.tflite

In [11]:
with open(tflite_model_path, "rb") as f:
    tflite_model = f.read()

print("✅ TFLite model loaded, size:", len(tflite_model)/1024, "KB")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Dishang\\Documents\\Arduino\\tiny_ml\\gender_model.tflite'

In [12]:
import os

# Path to your TFLite model
tflite_model_path = r"C:\Users\Dishang\Documents\Arduino\tiny_ml\tiny_ml\gender_model.tflite"

# Load the TFLite model
with open(tflite_model_path, "rb") as f:
    tflite_model = f.read()

print("✅ TFLite model loaded, size:", len(tflite_model)/1024, "KB")

# Path to save the generated C header file
header_path = r"C:\Users\Dishang\Documents\Arduino\tiny_ml\gender_model.h"
folder_path = os.path.dirname(header_path)

# Ensure folder exists
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("✅ Folder created:", folder_path)
else:
    print("✅ Folder exists:", folder_path)

# Function to convert TFLite bytes to C array
def tflite_to_c_array(tflite_bytes, array_name="model_data"):
    c_code = f"const unsigned char {array_name}[] = {{\n"
    for i, b in enumerate(tflite_bytes):
        if i % 12 == 0:
            c_code += "  "
        c_code += f"0x{b:02x}, "
        if (i + 1) % 12 == 0:
            c_code += "\n"
    c_code += "\n};\n"
    c_code += f"const unsigned int {array_name}_len = {len(tflite_bytes)};\n"
    return c_code

# Convert and save to header file
try:
    with open(header_path, "w") as f:
        f.write(tflite_to_c_array(tflite_model, "gender_model_data"))
    print("✅ Header file created:", header_path)
    print("Header file size (KB):", os.path.getsize(header_path)/1024)
except Exception as e:
    print("❌ Error writing header file:", e)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Dishang\\Documents\\Arduino\\tiny_ml\\tiny_ml\\gender_model.tflite'

In [77]:
tflite_model_path = r"C:\tiny_ml\gender_model.tflite"
header_path = r"C:\tiny_ml\gender_model.h"
os.makedirs(os.path.dirname(header_path), exist_ok=True)


In [78]:
tflite_model_path = r"C:\Users\Dishang\Documents\Arduino\tiny_ml\gender_model.tflite"
header_path = r"C:\tiny_ml\gender_model.h"  # This is where the header will be created
os.makedirs(os.path.dirname(header_path), exist_ok=True)


In [79]:
# Load the TFLite model
with open(tflite_model_path, "rb") as f:
    tflite_model = f.read()

print("✅ TFLite model loaded, size:", len(tflite_model)/1024, "KB")

# Convert to C array
c_array_code = tflite_to_c_array(tflite_model, "gender_model_data")

# Save header
with open(header_path, "w") as f:
    f.write(c_array_code)

print("✅ Header file created:", header_path)
print("Header file size (KB):", os.path.getsize(header_path)/1024)


✅ TFLite model loaded, size: 166.390625 KB
✅ Header file created: C:\tiny_ml\gender_model.h
Header file size (KB): 1053.9072265625


In [50]:
import tensorflow as tf
import numpy as np

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Count total parameters
total_params = 0
for detail in interpreter.get_tensor_details():
    shape = detail['shape']
    total_params += np.prod(shape)

print(f"Approximate total parameters in TFLite model: {total_params}")


Approximate total parameters in TFLite model: 1070117


In [35]:
import cv2

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

cap = cv2.VideoCapture(0)  # webcam

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    for (x,y,w,h) in faces:
        face_img = frame[y:y+h, x:x+w]
        face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
        gender, conf = predict_gender_tflite(face_rgb)

        # Draw rectangle and label
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
        cv2.putText(frame, f"{gender} ({conf*100:.1f}%)", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    cv2.imshow("TFLite Gender Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
